# Caleb Wilson I513 Homework 5

In [1]:
#import necessary modules
import pandas as pd
import numpy as np
import sqlite3
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_validate

## Connecting to SQL Database and Building Pandas Dataframe

In [2]:
#connect python to the sqlite database
dbPath = "database.sqlite"
connection = sqlite3.connect(dbPath)
cursor = connection.cursor()

In [3]:
#read the sql data base into a pandas data frame
player_attri_df = pd.read_sql("SELECT * FROM Player_Attributes", connection)

#fill in nan and null variables
player_attri_df["gk_reflexes"].fillna(player_attri_df["gk_reflexes"].mean(), inplace=True)
player_attri_df["gk_handling"].fillna(player_attri_df["gk_handling"].mean(), inplace=True)


In [4]:
#validate that there are no null values
player_attri_df["gk_reflexes"].isnull().any()

False

In [5]:
#validate that there are no nan values
player_attri_df["gk_reflexes"].isna().any()

False

In [6]:
#validate that there are no null values
player_attri_df["gk_handling"].isnull().all()

False

In [7]:
#validate that there are no nan values
player_attri_df["gk_handling"].isna().all()

False

In [8]:
#check out what the dataframe looks like
player_attri_df.head(10)

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
5,6,189615,155782,2016-04-21 00:00:00,74.0,76.0,left,high,medium,80.0,...,66.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
6,7,189615,155782,2016-04-07 00:00:00,74.0,76.0,left,high,medium,80.0,...,66.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
7,8,189615,155782,2016-01-07 00:00:00,73.0,75.0,left,high,medium,79.0,...,65.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
8,9,189615,155782,2015-12-24 00:00:00,73.0,75.0,left,high,medium,79.0,...,65.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
9,10,189615,155782,2015-12-17 00:00:00,73.0,75.0,left,high,medium,79.0,...,65.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0


In [9]:
player_attri_df.columns.get_loc('gk_handling')

38

In [10]:
player_attri_df.columns.get_loc('gk_reflexes')

41

## Setting up Variables and Datasets

In [11]:
# create x and y variables from the data frame read from sql for machine learning analysis
x = player_attri_df["gk_reflexes"].values
y = player_attri_df["gk_handling"].values

#check out what the data looks like
print(x.shape, y.shape)

###x, y = normalize(player_attri_df["gk_reflexes"]), player_attri_df["gk_handling"]

(183978,) (183978,)


In [12]:
#data segmentation with training and test datasets
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.3)

#reshape the array
X_train = np.array(X_train).reshape(-1,1)
X_test = np.array(X_test).reshape(-1,1)

###Y_train = np.array(Y_train).reshape(-1,1)
###Y_test = np.array(Y_test).reshape(-1,1)


## Linear Regression

In [13]:
#linear regression, fits model based on X_train and Y_train values
linear = LinearRegression()

#train the linear regression data
linear.fit(X_train, Y_train)

#print out linear regressions statistics
print('Slope: %.3f'%linear.coef_[0])
print('Intercept: %.3f'%linear.intercept_)

Slope: 0.891
Intercept: 1.434


In [14]:
#test data
#Get the predicted Y result (pred_linear) through the model based on the X_test value
Y_test_predicted = linear.predict(X_test)

###Y_train_predicted = linear.predict(X_train)

In [15]:
#use R2 score to find the correlation of the linear regression technique
rsquared = r2_score(Y_test_predicted, Y_test)
print("R2 Score: " + str(rsquared))

R2 Score: 0.9301614547443459


## Cross Validation

In [16]:
# set up scaler
scalar = StandardScaler()
# reshape data
x = np.array(x).reshape(-1,1)
x = np.array(x).reshape(-1,1)
y = np.array(y).reshape(-1,1)
y = np.array(y).reshape(-1,1)
# transform data
x = scalar.fit_transform(x)
y = scalar.fit_transform(y)

In [17]:
#data segmentation with training and test datasets
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.3)

#linear regression, fits model based on X_train and Y_train values
linear = LinearRegression()

#train the linear regression data
###linear.fit(X_train, Y_train)

#reshape the array
###X_train = np.array(X_train).reshape(-1,1)
###X_test = np.array(X_test).reshape(-1,1)

###Y_train = np.array(Y_train).reshape(-1,1)
###Y_test = np.array(Y_test).reshape(-1,1)


In [18]:
#cross validation 

# using linear to fit x_train, y_trainwith 10-fold cross-validation
cross_scores = cross_val_score(linear, X_train,Y_train,cv = 10)

In [19]:
# average R^2 scores
print(cross_scores.mean())

0.9340603518303083


In [20]:
#use R2 score to find the correlation of the cross validation technique
###rsquared = r2_score(Y_test_predicted, Y_test)
print("R2 Score: " + str(cross_scores.mean()))

R2 Score: 0.9340603518303083


## Ridge Regularization

In [21]:
# set up scaler
scalar = StandardScaler()
# reshape data
x = np.array(x).reshape(-1,1)
x = np.array(x).reshape(-1,1)
y = np.array(y).reshape(-1,1)
y = np.array(y).reshape(-1,1)
# transform data
x = scalar.fit_transform(x)
y = scalar.fit_transform(y)

In [22]:
#data segmentation with training and test datasets
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.3)

#linear regression, fits model based on X_train and Y_train values
#linear = LinearRegression()

#train the linear regression data
###linear.fit(X_train, Y_train)

#reshape the array
###X_train = np.array(X_train).reshape(-1,1)
###X_test = np.array(X_test).reshape(-1,1)

###Y_train = np.array(Y_train).reshape(-1,1)
###Y_test = np.array(Y_test).reshape(-1,1)


In [23]:
#ridge regularization

# using ridge to fit X_train, Y_train with 10-fold cross-validation over alpha values from 0.1 to 0.9
for i in range (1,10):
    ridge = Ridge(alpha=i/10)
    ridge_scores = cross_val_score(ridge, X_train, Y_train, cv=10)
    print(i/100, ": ", ridge_scores.mean())

0.01 :  0.93339888301785
0.02 :  0.9333988830426172
0.03 :  0.9333988830659795
0.04 :  0.9333988830879367
0.05 :  0.933398883108489
0.06 :  0.933398883127636
0.07 :  0.933398883145378
0.08 :  0.9333988831617152
0.09 :  0.9333988831766471


In [24]:
#use R2 score to find the correlation of the ridge regularization technique
###rsquared = r2_score(Y_test_predicted, Y_test)

print("R2 Score: " + str(ridge_scores.mean()))

R2 Score: 0.9333988831766471


## Lasso Regularization

In [25]:
# set up scaler
scalar = StandardScaler()
# reshape data
x = np.array(x).reshape(-1,1)
x = np.array(x).reshape(-1,1)
y = np.array(y).reshape(-1,1)
y = np.array(y).reshape(-1,1)
# transform data
x = scalar.fit_transform(x)
y = scalar.fit_transform(y)

In [26]:
#data segmentation with training and test datasets
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.3)

#linear regression, fits model based on X_train and Y_train values
#linear = LinearRegression()

#train the linear regression data
###linear.fit(X_train, Y_train)

#reshape the array
###X_train = np.array(X_train).reshape(-1,1)
###X_test = np.array(X_test).reshape(-1,1)

###Y_train = np.array(Y_train).reshape(-1,1)
###Y_test = np.array(Y_test).reshape(-1,1)


In [27]:
#lasso regularization

# using lasso to fit X_train, Y_train with 10-fold cross-validation over alpha values from 0.1 to 0.9
for i in range(1,10):
    lasso = Lasso(alpha=i/10)
    lasso_scores = cross_val_score(lasso, X_train, Y_train, cv=10)
    print(i/100, ": ", lasso_scores.mean())

0.01 :  0.9238311458018114
0.02 :  0.8939939730878834
0.03 :  0.8442633808534158
0.04 :  0.7746393690984082
0.05 :  0.6851219378228611
0.06 :  0.5757110870267745
0.07 :  0.44640681671014804
0.08 :  0.29720912687298184
0.09 :  0.12811801751527613


In [28]:
#use R2 score to find the correlation of the lasso regularization technique
###rsquared = r2_score(Y_test_predicted, Y_test)

print("R2 Score: " + str(lasso_scores.mean()))

R2 Score: 0.12811801751527613


## Other attempted methods:

In [29]:
#from sklearn.model_selection import cross_validate
#from sklearn import datasets, linear_model
#from sklearn.metrics import make_scorer
#from sklearn.metrics import confusion_matrix
#from sklearn.svm import LinearSVC
#from sklearn import svm

In [30]:
#correlation through corr() and preformance comparison with r2_score() shows how much difference is between the pred_linear value and the actual y_test value

# correlation
#corr_linear= pd.Series(Y_test_predicted).corr(pd.Series(Y_test))
# r2_score
#rsquared_linear= r2_score(Y_test_predicted, Y_test)

In [31]:
#the correlation value
#print(corr_linear)

In [32]:
#the rsquared value
#print(rsquared_linear)

In [33]:
#the linear coefficient 
#print(linear.coef_)

In [34]:
#model performance evaluation
#print('R^2 train: %.3f, test %.3f' %(r2_score(Y_train, Y_train_predicted),r2_score(Y_test, Y_test_predicted)))

In [35]:
#corr_linear = pd.Series(Y_predicted).corr(pd.Series(Y_test))

In [36]:
#clf = svm.SVC(kernel='linear', C=1).fit(X_train, Y_train)
#clf.score(X_test, Y_test)

In [37]:
#lasso = linear_model.Lasso()

In [38]:
#cv_results = cross_validate(lasso, x, y, cv = 2, scoring=('r2','neg_mean_squared_error'), return_train_score=True)
#print(scores['test_neg_mean_squared_error'])

In [39]:
#print(scores['train_r2'])

In [40]:
#coefs = []
# using ridge to fit X_train, y_train with 10-fold cross-validation over alpha values
#for i, alpha in enumerate([0.1, 0.5, 1, 5, 10, 20, 50, 100]):
    # adding new df for alpha results to list
    #coefs.append(pd.DataFrame(columns=player_attri_df.loc[:,'gk_reflexes']))
    # coefs.append(pd.DataFrame(columns=player_attri_df.loc[:,'gk_handling']))
    # coefs_average = []
    #ridge_scores = []
    #ridge = Ridge(alpha=alpha)
    
    #cv_results = cross_validate(ridge, X_train, Y_train, cv=10, return_estimator=True)
    
    #ridge_scores = cross_val_score(ridge, X_train, Y_train, cv=10)

    # compute average coef value over k-fold per feature 
    #for j in cv_results['estimator']:
    #    coefs[i].loc[j]=j.coef_
    # create list of average coefs
    #for column in coefs[i]:
    #    coefs_average.append(coefs[i][column].mean())

In [41]:
#coefs = []
# using lasso to fit X_train, y_train with 10-fold cross-validation over alpha values
#for i, alpha in enumerate([0.1, 0.5, 1, 5, 10, 20, 50, 100]):
    # adding new df for alpha results to list
   # coefs.append(pd.DataFrame(columns=df.columns[:-1]))
    #coefs_average = []
    #lasso_scores = []
    
   # lasso = Lasso(alpha=alpha)
   # cv_results = cross_validate(lasso, X_train, y_train, cv=10, return_estimator=True)
   # lasso_scores = cross_val_score(lasso, X_train, y_train, cv=10)

    # compute average coef value over k-fold per feature 
   # for j in cv_results['estimator']:
   #     coefs[i].loc[j]=j.coef_
    # create list of average coefs
  #  for column in coefs[i]:
  #      coefs_average.append(coefs[i][column].mean())